In [7]:
import pulp

In [8]:
FUEL_TANK_CAPACITY = 12000
MAX_FUEL_PURCHASE = 10000
FUEL_SAFETY_MARGIN = 600
CITIES = ['Los Angeles', 'Houston', 'New York', 'Miami']
outbound_segment_distance = [1500, 1700, 1300, 2700]
local_fuel_price = [0.88, 0.15, 1.05, 0.95]

In [9]:
prob = pulp.LpProblem("Airplane_Fuel_Optimization", pulp.constants.LpMinimize)

In [11]:
# Set up the decision variables
# Amount of fuel purchased at each city
fuel_purchased = pulp.LpVariable.dicts("Fuel_Purchased", 
                                     CITIES,
                                     lowBound=0,
                                     upBound=MAX_FUEL_PURCHASE)

# Fuel level at landing at each city
fuel_at_landing = pulp.LpVariable.dicts("Fuel_At_Landing",
                                       CITIES,
                                       lowBound=FUEL_SAFETY_MARGIN,
                                       upBound=FUEL_TANK_CAPACITY)

# Fuel level at takeoff from each city
fuel_at_takeoff = pulp.LpVariable.dicts("Fuel_At_Takeoff",
                                       CITIES,
                                       lowBound=0,
                                       upBound=FUEL_TANK_CAPACITY)

In [12]:
# Constraints
for city in CITIES:
    # 1. Fuel at takeoff equals landing fuel plus purchased fuel
    prob += fuel_at_takeoff[city] == fuel_at_landing[city] + fuel_purchased[city], \
            f"Fuel_Balance_{city}"
    
    # 2. Landing fuel must be at least safety margin
    prob += fuel_at_landing[city] >= FUEL_SAFETY_MARGIN, \
            f"Safety_Margin_{city}"
    
    # 3. Purchased fuel must not exceed maximum purchase
    prob += fuel_purchased[city] <= MAX_FUEL_PURCHASE, \
            f"Max_Purchase_{city}"
    
    # 4. Fuel at takeoff must not exceed tank capacity
    prob += fuel_at_takeoff[city] <= FUEL_TANK_CAPACITY, \
            f"Tank_Capacity_{city}"

In [ ]:
# Fuel Consumption Constraints - Linearized
for i in range(len(CITIES)):
    from_city = CITIES[i]
    to_city = CITIES[(i + 1) % len(CITIES)]  # Next city (wraps around)
    distance = outbound_segment_distance[i]
    
    # (1 + (d/4000))*y = (1 - (d/4000))*z - d
    # where y = landing fuel (to_city), z = takeoff fuel (from_city), d = distance (from_city)
    
    prob += ((1 + distance/4000) * fuel_at_landing[to_city] == 
             (1 - distance/4000) * fuel_at_takeoff[from_city] - distance), \
            f"Fuel_Consumption_{from_city}_to_{to_city}"
    
# Test city pairs and distances
print("Testing city connections and distances:")
for i in range(len(CITIES)):
    from_city = CITIES[i]
    to_city = CITIES[(i + 1) % len(CITIES)]
    distance = outbound_segment_distance[i]
    
    print(f"From {from_city:12} to {to_city:12} - Distance: {distance:4} miles")

Testing city connections and distances:
From Los Angeles  to Houston      - Distance: 1500 miles
From Houston      to New York     - Distance: 1700 miles
From New York     to Miami        - Distance: 1300 miles
From Miami        to Los Angeles  - Distance: 2700 miles


In [14]:
# Objective Function: Minimize total fuel cost
prob += pulp.lpSum([fuel_purchased[city] * local_fuel_price[i] 
                    for i, city in enumerate(CITIES)]), "Total_Fuel_Cost"

In [15]:
# Solve the optimization problem
prob.solve()

# Print status
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Optimal Cost: ${pulp.value(prob.objective):,.2f}")

# Print results for each city
for i, city in enumerate(CITIES):
    print(f"\n{city}:")
    print(f"  Fuel Purchased: {fuel_purchased[city].value():,.1f} gallons")
    print(f"  Fuel at Takeoff: {fuel_at_takeoff[city].value():,.1f} gallons")
    print(f"  Fuel at Landing: {fuel_at_landing[city].value():,.1f} gallons")
    print(f"  Local Price: ${local_fuel_price[i]:.2f}/gallon")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/envs/build-the-death-star/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ty/f5d3n0vj42jdwf7h9rrgs5gh0000gn/T/be80dc0582e34c69bc6c355519363270-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ty/f5d3n0vj42jdwf7h9rrgs5gh0000gn/T/be80dc0582e34c69bc6c355519363270-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 62 RHS
At line 83 BOUNDS
At line 100 ENDATA
Problem MODEL has 20 rows, 12 columns and 32 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-16) rows, 8 (-4) columns and 12 (-20) elements
0  Obj 9773.0693 Primal inf 10078.158 (4)
5  Obj 15414.957
Optimal - objective value 15414.957
After Postsolve, objective 15414.957, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 15414.95673 - 5 iterations time 0.002, Pre

Looking at the simulation results above, my first instinct is to ask, "Why not land in Miami with the minimum fuel and buy fuel 10c / gallon cheaper?". Adding that constraint and running the model again to scratch that itch.

In [16]:
# Add constraint to fix landing fuel in Miami
prob += fuel_at_landing['Miami'] == 600, "Fix_Miami_Landing"

# Re-solve
prob.solve()

# Print status
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Optimal Cost: ${pulp.value(prob.objective):,.2f}")

# Print results for each city
for i, city in enumerate(CITIES):
    print(f"\n{city}:")
    print(f"  Fuel Purchased: {fuel_purchased[city].value():,.1f} gallons")
    print(f"  Fuel at Takeoff: {fuel_at_takeoff[city].value():,.1f} gallons")
    print(f"  Fuel at Landing: {fuel_at_landing[city].value():,.1f} gallons")
    print(f"  Local Price: ${local_fuel_price[i]:.2f}/gallon")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/envs/build-the-death-star/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ty/f5d3n0vj42jdwf7h9rrgs5gh0000gn/T/b8c435308ed84e5dbe96a79e9317ea19-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ty/f5d3n0vj42jdwf7h9rrgs5gh0000gn/T/b8c435308ed84e5dbe96a79e9317ea19-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 26 COLUMNS
At line 64 RHS
At line 86 BOUNDS
At line 103 ENDATA
Problem MODEL has 21 rows, 12 columns and 33 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 14975.489 (8)
9  Obj 15414.957 Primal inf 946.9303 (1)
Primal infeasible - objective value 15414.957
PrimalInfeasible objective 15414.95673 - 9 iterati

The current simulation is probably not able to make this solution work because of the circular nature of the constraints. However, in a situation where we can directly estimate fuel consumption for any given segment, I imagine the value could be calculated.